In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Load the pickled data without the new individuals added in the previous notebook.

G = nx.read_gpickle('Synthetic Social Network.pkl')

# Hubs: How do we evaluate the importance of some individuals in a network?

Within a social network, there will be certain individuals which perform certain important functions. For example, there may be hyper-connected individuals who are connected to many, many more people. They would be of use in the spreading of information. Alternatively, if this were a disease contact network, identifying them would be useful in stopping the spread of diseases. How would one identify these people?

## Approach 1: Neighbors

One way we could compute this is to find out the number of people an individual is conencted to. NetworkX let's us do this by giving us a `G.neighbors(node)` function.

In [ ]:
# Let's find out the number of neighbors that individual #7 has.
G.neighbors(9)

### Exercise

Can you create a ranked list of the importance of each individual, based on the number of neighbors they have? 

Hint: One suggested output would be a list of tuples, where the first element in each tuple is the node ID (an integer number), and the second element is a list of its neighbors.

Hint: Python's `sorted(iterable, key=lambda x:...., reverse=True)` function may be of help here.

## Approach 2: Degree Centrality

The number of other nodes that one node is connected to is a measure of its centrality. NetworkX implements a **degree centrality**, which is defined as the number of neighbors that a node has normalized to the number of individuals it could be connected to in the entire graph. This is accessed by using `nx.degree_centrality(G)`

In [ ]:
nx.degree_centrality(G)

If you inspect the dictionary closely, you will find that node 19 is the one that has the highest degree centrality, just as we had measured by counting the number of neighbors.

There are other measures of centrality, namely **betweenness centrality**, **flow centrality** and **load centrality**. You can take a look at their definitions on the NetworkX API docs and their cited references. You can also define your own measures if those don't fit your needs, but that is an advanced topic that won't be dealt with here.

The NetworkX API docs that document the centrality measures are here: http://networkx.readthedocs.io/en/networkx-1.11/reference/algorithms.centrality.html?highlight=centrality#module-networkx.algorithms.centrality

### Exercises

1. Can you create a histogram of the distribution of degree centralities?
2. Can you create a histogram of the distribution of number of neighbors?
3. Can you create a scatterplot of the degree centralities against number of neighbors?
4. If I have `n` nodes, then how many possible edges are there in total, assuming self-edges are allowed? What if self-edges are not allowed?

Hint: You may want to use:

    plt.hist(list_of_values)

and 

    plt.scatter(x_values, y_values)

If you know the Matplotlib API, feel free to get fancy :).

In [ ]:
# Answer to question 1

In [ ]:
# Answer to question 2

In [ ]:
# Answer to question 3

# Paths in a Network

Graph traversal is akin to walking along the graph, node by node, restricted by the edges that connect the nodes. Graph traversal is particularly useful for understanding the local structure (e.g. connectivity, retrieving the exact relationships) of certain portions of the graph and for finding paths that connect two nodes in the network. 

Using the synthetic social network, we will figure out how to answer the following questions:

1. How long will it take for a message to spread through this group of friends? (making some assumptions, of course)
2. How do we find the shortest path to get from individual A to individual B?

## Shortest Path

In [ ]:
nx.draw(G, with_labels=True)

Let's say we wanted to find the shortest path between two nodes. How would we approach this? One approach is what one would call a **breadth-first search** (http://en.wikipedia.org/wiki/Breadth-first_search). While not necessarily the fastest, it is the easiest to conceptualize. 

The approach is essentially as such:

1. Begin with a queue of the starting node.
2. Add the neighbors of that node to the queue.
    1. If destination node is present in the queue, end.
    2. If destination node is not present, proceed.
3. For each node in the queue:
    1. Remove node from the queue.
    2. Add neighbors of the node to the queue. Check if destination node is present or not.
    3. If destination node is present, repeat Step 3.
    4. If destination node is not present, continue.

### Exercise

Try implementing this algorithm in a function called `path_exists(node1, node2, G)`.

The function should take in two nodes, `node1` and `node2`, and the graph `G` that they belong to, and return a Boolean that indicates whether a path exists between those two nodes or not. For convenience, also print out whether a path exists or not between the two node.s

In [ ]:
def path_exists(node1, node2, G):
    """
    This function checks whether a path exists between two nodes (node1, node2) in graph G.
    
    Special thanks to @ghirlekar for suggesting that we keep track of the "visited nodes" to
    prevent infinite loops from happening. 
    
    Reference: https://github.com/ericmjl/Network-Analysis-Made-Simple/issues/3

    """
    # Fill in code below
    queue = [______]
    destination = _______
    visited_nodes = set()
    
    for _____ in ______:
        ______ = G.neighbors(_____)
        if _______ in ______:
            print('Path exists between nodes {0} and {1}'.format(node1, node2))
            return True
            break
        else:
            _______.remove(_______)
            visited_nodes.add(______)
            _______.extend(_______)
        if len(_______) == ____:
            print('Path does not exist between nodes {0} and {1}'.format(node1, node2))
            return False
    

And testing the function on a few test cases:

- 18 and any other node (should return `False`)
- 29 and 26 (should return `True`)

In [ ]:
path_exists(18, 5, G)
path_exists(29, 26, G)

If you write an algorithm that runs breadth-first, the recursion pattern is likely to follow what we have done above. If you do depth-first, the recursion pattern is likely to look a bit different. We can figure that out together later, if there is time.

Meanwhile... thankfully, NetworkX has a function for us to use, titled `has_path`, so we don't have to implement this on our own. :-)

http://networkx.readthedocs.io/en/networkx-1.11/reference/generated/networkx.algorithms.shortest_paths.generic.has_path.html

In [ ]:
nx.has_path(G, 18, 5)

NetworkX also has other shortest path algorithms implemented. 

http://networkx.readthedocs.io/en/networkx-1.11/reference/algorithms.shortest_paths.html

We can build upon these to build our own graph query functions. Let's see if we can trace the shortest path from one node to another.

`nx.shortest_path(G, source, target)` gives us a list of nodes that exist within one of the shortest paths between the two nodes. (Not all paths are guaranteed to be found.)

In [ ]:
nx.shortest_path(G, 4, 14)

Incidentally, the node list is in order as well - we will travel through 19 and 17 in that order to get from 14 from 4.

### Exercise

Write a function that extracts the edges in the shortest path between two nodes and puts them into a new graph, and draws it to the screen. It should also return an error if there is no path between the two nodes.

Hint: You may want to use `G.subgraph(iterable_of_nodes)` to extract just the nodes and edges of interest from the graph `G`. You might want to use the following line of code somewhere:

    newG = G.subgraph(nodes_of_interest)
    
`newG` will be a new graph, of the same type as `G`, comprised of the nodes of interest and the edges that connect them.

In [ ]:
# Possible Answer:

def extract_path_edges(G, source, target):
    # Check to make sure that a path does exists between source and target.
    
    # Fill in answer below.
        
newG = extract_path_edges(G, 1, 14)
nx.draw(newG, with_labels=True)

### Exercise

Since we've been drawing some graphs to screen, we might as well draw a few other things while we're on a roll.

Write a function that extracts only node, its neighbors, and the edges between that node and its neighbors as a new graph. Then, draw the new graph to screen.

In [ ]:
# Possible Answer

def extract_neighbor_edges(G, node):
    # Fill in code below
    
    
    return newG

fig = plt.figure(0)
newG = extract_neighbor_edges(G, 19)
nx.draw(newG, with_labels=True)

In [ ]:
def extract_neighbor_edges2(G, node):
    # Fill in code below
    
    
    return newG

fig = plt.figure(1)
newG = extract_neighbor_edges2(G, 19)
nx.draw(newG, with_labels=True)

### Challenge Exercises (optional)

Let's try some other problems that build on the NetworkX API.

Refer to the following for the relevant functions:

http://networkx.readthedocs.io/en/networkx-1.11/reference/algorithms.shortest_paths.html

1. If we want a message to go from one person to another person, and we assume that the message takes 1 day for the initial step and 1 additional day per step in the transmission chain (i.e. the first step takes 1 day, the second step takes 2 days etc.), how long will the message take to spread from any two given individuals? Write a function to compute this.
2. What is the distribution of message spread times from person to person? What about chain lengths?
3. Are there certain individuals who consistently show up in the chain? (Hint: you might wish to use the following functions/objects:
    1. `Counter` object from the `collections` module 
    2. `combinations` function from the `itertools` module.
    3. `all_shortest_paths(G, node1, node2)` which is part of the networkX algorithms.
4. As a bonus, if you were able to compute the answer to question 3, can you plot a histogram of the number of times each node shows up in a connecting path?

In [ ]:
# Hint to Question 1:
# All we need here is the length of the path.

def compute_transmission_time(G, source, target):
    """
    Fill in code below.
    """

    return time

compute_transmission_time(G, 14, 4)       

In [ ]:
# Hint to Question 2:
# We need to know the length of every single shortest path between every pair of nodes.
# If we don't put a source and target into the nx.shortest_path_length(G) function call, then
# we get a dictionary of dictionaries, where all source-->target-->lengths are shown.

# Fill in code below:






In [ ]:
# Fill in code below for Question 3:


In [ ]:
# Hint to Question 4:
# Use plt.bar()


# Hubs Revisited

It looks like individual 19 is an important person of some sorts - if a message has to be passed through the network in the shortest time possible, then usually it'll go through person 19. Such a person has a high **betweenness centrality**. This is implemented as one of NetworkX's centrality algorithms. Check out the Wikipedia page for a further description.

http://en.wikipedia.org/wiki/Betweenness_centrality

In [ ]:
btws = nx.betweenness_centrality(G, normalized=False)
plt.bar(btws.keys(), btws.values())

### Exercise

Plot betweeness centrality against degree centrality for the synthetic social network above.

In [ ]:
# Possible answer:
deg_cent_sorted = _______
btw_cent_sorted = _______




# Make the plot
plt.scatter(deg_cent_sorted, btw_cent_sorted)

**Think about it...**

From the scatter plot, we can see that the dots don't all fall on the same line. Degree centrality and betweenness centrality don't necessarily correlate. Can you think of a reason why?

What would be the degree centrality and betweenness centrality of the middle connecting node in the **barbell graph** below?

In [ ]:
nx.draw(nx.barbell_graph(5, 1))